# Industry 4.0 의 중심, BigData

<div align='right'><font size=2 color='gray'>Data Processing Based Python @ <font color='blue'><a href='https://www.facebook.com/jskim.kr'>FB / jskim.kr</a></font>, [김진수](bigpycraft@gmail.com)</font></div>
<hr>

## 부품가격 조회사이트에서 필요한 정보 파싱
- <a href='http://221.143.43.214/'>스카니아 부품가격 조회</a>
- 브랜드별 SCANIA, HINO 검색정보 리스트 가져오기
- 컬럼정보 : 적용일, 순번, 브랜드, 부품그룹, 부품번호, 부품명(한글), 부품명(영문), 가격(VAT별도)


## 스카니아 부품가격정보 크롤링

In [ ]:
from selenium import webdriver as wd
from bs4 import BeautifulSoup as bs

from selenium.webdriver.common.by import By

# 명시적 대기를 위해
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from IPython.display import Image

import numpy as np
import pandas as pd
import platform
import time
import sys

### <font color='brown'>Step1. Device Info 가져오기</font>

In [10]:
# 드라이브 로드
if platform.system() == 'Darwin':    # MacOS
    driver = wd.Chrome(executable_path='./driver/chromedriver')      
elif platform.system() == 'Windows': # Windows
    driver = wd.Chrome(executable_path='./driver/chromedriver.exe')    
else:
    print("It's unknown system. Hangul fonts are not supported!")

In [ ]:
target_url = 'http://221.143.43.214/'
brend_keyword = 'SCANIA'    # SCANIA or HINO

In [ ]:
# 사이트 접속 (get) : 셀레니움이 하는 일은 사림이 하는일과 최대한 유사하게 동작하도록 작업
# time.sleep(2)
driver.get(target_url)

In [ ]:
capture_img = './images/scania_site.png'
driver.save_screenshot(capture_img)
Image(capture_img)

In [ ]:
html = driver.page_source
soup = bs(html, "lxml")
soup.title

In [ ]:
brend_keyword = 'SCANIA'    # SCANIA or HINO
driver.find_element_by_css_selector('#brend').send_keys(brend_keyword)

In [ ]:
# 검색 버튼 클릭
search_btn = '#con2 > div > div.search2 > form > fieldset > table > tbody > tr:nth-child(3) > td > button'
driver.find_element_by_css_selector(search_btn).click()

In [ ]:
html = driver.page_source
soup = bs(html, "lxml")
soup.title

In [ ]:
result_cnt = soup.find('span', 'fcop4').get_text()
result_cnt = int(result_cnt)
result_cnt

In [ ]:
last_page = int(result_cnt / 15)
last_page

In [11]:
def get_total_count(brand='SCANIA'):
    
    target_url = 'http://221.143.43.214/'
    brend_keyword = brand    # SCANIA or HINO

    driver.get(target_url)

    html = driver.page_source
    soup = bs(html, "lxml")
    
    # 검색 브랜드 선택
    driver.find_element_by_css_selector('#brend').send_keys(brend_keyword)

    # 검색 버튼 클릭
    search_btn = '#con2 > div > div.search2 > form > fieldset > table > tbody > tr:nth-child(3) > td > button'
    driver.find_element_by_css_selector(search_btn).click()

    html = driver.page_source
    soup = bs(html, "lxml")

    result_cnt = soup.find('span', 'fcop4').get_text()
    result_cnt = int(result_cnt)

    last_page = int(result_cnt / 15)
    
    print('검색된 총 데이터수는 {}건이며, 마지막 페이지는 {}입니다.'.format(result_cnt, last_page))
    
    return result_cnt, last_page



In [12]:
result_cnt, last_page = get_total_count()

검색된 총 데이터수는 24525건이며, 마지막 페이지는 1635입니다.


In [ ]:
page_url = 'http://221.143.43.214/index.html?page={}&key=Y&brend={}&series2=&&series3=&part_no=&type1=&type2='
param_page  = last_page
param_brend = 'SCANIA'

In [ ]:
driver.get(page_url.format(1, param_brend))

In [ ]:
capture_img = './images/scania_site_1.png'
driver.save_screenshot(capture_img)
Image(capture_img)

In [ ]:
html = driver.page_source
soup = bs(html, "lxml")
soup.title

In [ ]:
table_tag = soup.find('table', 'tbl2')
table_tag

In [ ]:
tbody_tag = table_tag.find('tbody')
# tbody_tag

In [ ]:
tr_tags = tbody_tag.find_all('tr')
tr_cnt = len(tr_tags)
tr_cnt

In [ ]:
td_tags = tr_tags[0].find_all('td')
td_cnt = len(td_tags)
td_cnt

In [ ]:
# 순번, ord_no
td_tags_0 = td_tags[0].get_text()
td_tags_0 = td_tags_0.strip()
td_tags_0

In [ ]:
# 브랜드, brand
td_tags_1 = td_tags[1].get_text()
td_tags_1 = td_tags_1.strip()
td_tags_1

In [ ]:
# 부품그룹, device_grp
td_tags_2 = td_tags[2].get_text()
td_tags_2 = td_tags_2.strip()
td_tags_2

In [ ]:
# 부품번호, device_no
td_tags_3 = td_tags[3].get_text()
td_tags_3 = td_tags_3.strip()
td_tags_3

In [ ]:
# 부품명(한글), device_nm_kr
td_tags_4 = td_tags[4].get_text()
td_tags_4 = td_tags_4.strip()
td_tags_4

In [ ]:
# 부품명(영문), device_nm_en
td_tags_5 = td_tags[5].get_text()
td_tags_5 = td_tags_5.strip()
td_tags_5

In [ ]:
# 가격(VAT별도), price
td_tags_6 = td_tags[6].get_text()
td_tags_6 = td_tags_6.replace(',', '')
td_tags_6 = td_tags_6.replace('\\', '')
td_tags_6 = td_tags_6.strip()
td_tags_6 = int(td_tags_6)
td_tags_6

In [ ]:
# 컬럼정보 : 순번, 브랜드, 부품그룹, 부품번호, 부품명(한글), 부품명(영문), 가격(VAT별도)
# Column : ord_no, brand, device_grp, device_no, device_nm_kr, device_nm_en, price

In [ ]:
def get_device_info(td_tags):
    device_info = list()
    td_tags_0 = td_tags[0].get_text().strip()    # 순번, ord_no
    td_tags_1 = td_tags[1].get_text().strip()    # 브랜드, brand
    td_tags_2 = td_tags[2].get_text().strip()    # 부품그룹, device_grp
    td_tags_3 = td_tags[3].get_text().strip()    # 부품번호, device_no
    td_tags_4 = td_tags[4].get_text().strip()    # 부품명(한글), device_nm_kr
    td_tags_5 = td_tags[5].get_text().strip()    # 부품명(영문), device_nm_en
    td_tags_6 = td_tags[6].get_text().strip()    # 가격(VAT별도), price
    td_tags_6 = td_tags_6.replace(',' , '')
    td_tags_6 = td_tags_6.replace('\\', '')
    td_tags_6 = td_tags_6.strip()
    
    # 숫자형
    td_tags_0 = int(td_tags_0)
    td_tags_6 = int(td_tags_6)
    
    device_info.append(td_tags_0)
    device_info.append(td_tags_1)
    device_info.append(td_tags_2)
    device_info.append(td_tags_3)
    device_info.append(td_tags_4)
    device_info.append(td_tags_5)
    device_info.append(td_tags_6)
    
    return device_info


In [ ]:
device_info = get_device_info(td_tags)
device_info

In [ ]:
def get_device_info(td_tags):
    device_info = dict()
    td_tags_0 = td_tags[0].get_text().strip()    # 순번, ord_no
    td_tags_1 = td_tags[1].get_text().strip()    # 브랜드, brand
    td_tags_2 = td_tags[2].get_text().strip()    # 부품그룹, device_grp
    td_tags_3 = td_tags[3].get_text().strip()    # 부품번호, device_no
    td_tags_4 = td_tags[4].get_text().strip()    # 부품명(한글), device_nm_kr
    td_tags_5 = td_tags[5].get_text().strip()    # 부품명(영문), device_nm_en
    td_tags_6 = td_tags[6].get_text().strip()    # 가격(VAT별도), price
    td_tags_6 = td_tags_6.replace(',', '')
    td_tags_6 = td_tags_6.replace('\\', '')
    td_tags_6 = td_tags_6.strip()
    
    # 숫자형
    td_tags_0 = int(td_tags_0)
    td_tags_6 = int(td_tags_6)
    
    
    device_info['ord_no'      ] = td_tags_0
    device_info['brand'       ] = td_tags_1
    device_info['device_grp'  ] = td_tags_2
    device_info['device_no'   ] = td_tags_3
    device_info['device_nm_kr'] = td_tags_4
    device_info['device_nm_en'] = td_tags_5
    device_info['price'       ] = td_tags_6
    
    return device_info


In [ ]:
device_info = get_device_info(td_tags)
device_info

### <font color='brown'>Step2. Device List 가져오기</font>

In [ ]:
tr_tags = tbody_tag.find_all('tr')
tr_cnt = len(tr_tags)
tr_cnt

In [ ]:
td_tags = tr_tags[0].find_all('td')
td_cnt = len(td_tags)
td_cnt

In [ ]:
def get_device_list(tr_tags):
    device_list = list()
    
    tr_cnt = len(tr_tags)
    if tr_cnt == 0:
        return 
    
    for i in range(tr_cnt):
        td_tags = tr_tags[i].find_all('td')
        device_info = get_device_info(td_tags)
        device_list.append(device_info)
    
    return device_list


In [ ]:
device_list = get_device_list(tr_tags)
device_list

### <font color='brown'>Step3. 페이지별 Device List 가져오기</font>

In [ ]:
def get_device_list(tr_tags):
    device_list = list()
    
    tr_cnt = len(tr_tags)
    if tr_cnt == 0:
        return None
    
    for i in range(tr_cnt):
        td_tags = tr_tags[i].find_all('td')
        device_info = get_device_info(td_tags)
        device_list.append(device_info)
    
    return device_list


In [ ]:
def get_device_list_by_page(start_pg, end_pg):
    device_list_total = list()
    
    page_url = 'http://221.143.43.214/index.html?page={}&key=Y&brend={}&series2=&&series3=&part_no=&type1=&type2='
    
    for navi_page in range(start_pg, end_pg+1):
        param_page  = navi_page
        param_brend = 'SCANIA'
        driver.get(page_url.format(param_page, param_brend))
        
        time.sleep(1)
        html = driver.page_source
        soup = bs(html, "lxml")
        
        table_tag = soup.find('table', 'tbl2')
        tbody_tag = table_tag.find('tbody')
        tr_tags = tbody_tag.find_all('tr')
        tr_cnt = len(tr_tags)
        
        if tr_cnt > 0:
            device_list = list()
        else:
            break
        
        device_list = get_device_list(tr_tags)
        
        device_list_total.extend(device_list)
    
    return device_list_total


In [ ]:
device_list_total = get_device_list_by_page(1, 10)
len(device_list_total)

In [ ]:
device_list_total

### <font color='brown'>Step4. 페이지별 Device List 저장하기</font>

In [ ]:
# 컬럼정보 : 순번, 브랜드, 부품그룹, 부품번호, 부품명(한글), 부품명(영문), 가격(VAT별도)
# Column : ord_no, brand, device_grp, device_no, device_nm_kr, device_nm_en, price

In [ ]:
import csv

csv_columns = [ 'ord_no', 'brand', 'device_grp', 'device_no', 'device_nm_kr', 'device_nm_en', 'price' ]
dict_data = [
     {'ord_no': 1,
      'brand': 'SCANIA',
      'device_grp': '13. 조임장치및가스켓',
      'device_no': '1752309',
      'device_nm_kr': '스터드',
      'device_nm_en': 'STUD',
      'price': 18600},
     {'ord_no': 2,
      'brand': 'SCANIA',
      'device_grp': '10. 캐빈관련',
      'device_no': '2082014',
      'device_nm_kr': '모델 뱃지',
      'device_nm_en': 'MODEL BADGE',
      'price': 66000},
     {'ord_no': 3,
      'brand': 'SCANIA',
      'device_grp': '3. 연료시스템',
      'device_no': '1851717',
      'device_nm_kr': '압력 조절장치',
      'device_nm_en': 'PRESSURE REGULATOR',
      'price': 114000},
]
# dict_data = device_list_total

csv_file = 'data/scania_test.csv'
try:
    with open(csv_file, 'w', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in dict_data:
            writer.writerow(data)
except IOError:
    print("I/O error") 
    

In [ ]:
df = pd.read_csv(csv_file)
df.head()

### <font color='brown'>Step5. 모듈기반 Scania 데이터 크롤링</font>

In [1]:
# from images import bigpycraft_dl as bpc
from module import bl_dc_scania as bpc
from IPython.display import Image 
from tqdm import tqdm_notebook

In [2]:
blScania = bpc.BlScania()
msg = blScania.chk_module('Scania Data Crawling !!')
msg

'Scania Data Crawling !!'

In [3]:
from selenium import webdriver as wd
from bs4 import BeautifulSoup as bs

from selenium.webdriver.common.by import By

# 명시적 대기를 위해
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from IPython.display import Image

import numpy as np
import pandas as pd
import platform
import time
import sys

In [4]:
# 드라이브 로드
if platform.system() == 'Darwin':    # MacOS
    driver = wd.Chrome(executable_path='./driver/chromedriver')      
elif platform.system() == 'Windows': # Windows
    driver = wd.Chrome(executable_path='./driver/chromedriver.exe')    
else:
    print("It's unknown system. Hangul fonts are not supported!")

In [5]:
key_brand='SCANIA'  # SCANIA or HINO
device_list_total = blScania.get_device_list_by_page(1000, 1005, driver, key_brand)
print('브랜드 {} 데이터 {}건이 크롤링 되었습니다!'.format(key_brand, len(device_list_total)))


브랜드 SCANIA 데이터 90건이 크롤링 되었습니다!


In [8]:
dirname  = 'data/'
filename = key_brand + '_' + str(len(device_list_total))
csv_file, msg = blScania.save_dict_to_csv(device_list_total, dirname, filename)
print(msg)
print(' File :', csv_file)

 성공적으로 저장되었습니다! 
 File : data/SCANIA_90.csv


In [9]:
df_scania = pd.read_csv(csv_file)
df_scania.tail(10)

,ord_no,brand,device_grp,device_no,device_nm_kr,device_nm_en,price
80,15066,SCANIA,11. 전기전자,1786943,케이블 하네스,CABLE HANESS,32600
81,15067,SCANIA,13. 조임장치및가스켓,1757465,프렌지 스크류,FRANGE SCREW,1000
82,15068,SCANIA,13. 조임장치및가스켓,1755682,프렌지 스크류,FRANGE SCREW,6100
83,15069,SCANIA,13. 조임장치및가스켓,1755674,프렌지 스크류,FRANGE SCREW,1100
84,15070,SCANIA,13. 조임장치및가스켓,1755673,프렌지 스크류,FRANGE SCREW,1000
85,15071,SCANIA,5. 동력전달장치,1754523,플러그,PLUG,36000
86,15072,SCANIA,13. 조임장치및가스켓,1754522,워셔,WASHER,2400
87,15073,SCANIA,5. 동력전달장치,1754521,워셔,WASHER,10600
88,15074,SCANIA,13. 조임장치및가스켓,1753179,플라스틱 파이프,PLASTIC PIPE,14100
89,15075,SCANIA,5. 동력전달장치,1752838,기어 샤프트,GEAR SH SHAFT,228000


### UTF-8 한글 CSV파일 Office에서 제대로 읽기 
> https://www.lesstif.com/pages/viewpage.action?pageId=54952504
- 엑셀을 실행하고 데이타 → 텍스트 를 선택합니다.
- 가져올 파일을 선택하고 확인을 클릭합니다.
- 콤보 박스에서 적절한 코드 타입이 선택됐는지 확인합니다.

In [6]:
dirname='data'
filename='scania_data.csv'

In [14]:
if not dirname.endswith('/'):
    dirname += '/'
    
if not filename.endswith('.csv'):
    filename += '.csv'

csv_file = dirname + filename
csv_file

'data/scania_data.csv'